In [ ]:
with open('/home/sahin/Downloads/kafka.txt','r') as doc:
    text=doc.read()
    

In [ ]:
char_to_num={char:i for i, char in enumerate(list(set(text)))}

In [ ]:
num_to_char={i:char for i ,char in enumerate(list(set(text)))}

In [ ]:
vocab_size=len(list(set(text)))
vocab_size

80

In [ ]:
num_to_char[5]

'U'

In [ ]:
char_to_num['h']

In [ ]:
import numpy as np
import sys
import os
import tensorflow
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import LSTM,Dense,Dropout
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard
import time
import datetime 

In [ ]:
seq_lenth=60

x_data=np.zeros((len(text)-1,seq_lenth,vocab_size ), dtype=np.int16)
y_data=np.zeros((len(text)-1,vocab_size), dtype=np.int16)

In [ ]:
x_list=[]
y_list=[]

for i in range(0,len(text)-seq_lenth,1):
    x_list.append(text[i:i+seq_lenth])
    y_list.append(text[i+seq_lenth])
    

In [ ]:
x_list[:5]

['One morning, when Gregor Samsa woke from troubled dreams, he',
 'ne morning, when Gregor Samsa woke from troubled dreams, he ',
 'e morning, when Gregor Samsa woke from troubled dreams, he f',
 ' morning, when Gregor Samsa woke from troubled dreams, he fo',
 'morning, when Gregor Samsa woke from troubled dreams, he fou']

In [ ]:
y_list[:5]

[' ', 'f', 'o', 'u', 'n']

In [ ]:
y_data[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int16)

In [ ]:
x_data[1,59]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int16)

In [ ]:
unix=time.time()
current_time= datetime.datetime.fromtimestamp(unix).strftime('%y_%m_%d_%H_%M_%S')

directory_1='/home/sahin/Desktop/checkspoints/{}'.format(current_time)
directory_2='/home/sahin/Desktop/logs/text_generation{}'.format(current_time)
tensorboard=TensorBoard(log_dir=directory_2)
model_check=ModelCheckpoint(filepath='check_parameters/{}'.format(current_time),monitor='loss',save_best_only=True,
                            save_weights_only=True, verbose=1)
if not os.path.exists(directory_1):
    os.makedirs(directory_1)
if not os.path.exists(directory_2):
    os.makedirs(directory_2)
    
model=Sequential()
model.add(LSTM(128, input_shape=(None,80)))
model.add(Dropout(0.2))
model.add(Dense(80,activation='softmax'))
optimizer=RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               107008    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 80)                10320     
Total params: 117,328
Trainable params: 117,328
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_data, y_data, batch_size=256, epochs=5, callbacks=[tensorboard,model_check])

Epoch 1/5
137627/137627 [==============================]137627/137627 [==============================] - 323s 2ms/step - loss: 0.0000e+00

Epoch 2/5
137627/137627 [==============================]137627/137627 [==============================] - 311s 2ms/step - loss: 0.0000e+00

Epoch 3/5
137627/137627 [==============================]137627/137627 [==============================] - 319s 2ms/step - loss: 0.0000e+00

Epoch 4/5
137627/137627 [==============================]137627/137627 [==============================] - 310s 2ms/step - loss: 0.0000e+00

Epoch 5/5
137627/137627 [==============================]137627/137627 [==============================] - 302s 2ms/step - loss: 0.0000e+00



In [ ]:
sequence=text[:60]
sys.stdout.write(sequence)

for k in range(0,200):
    x_pred=np.zeros((60,80),dtype=np.int16)
    for t, chars in enumerate(sequence):
        x_pred[t,char_to_num[chars]]=1
    
    x_pred=np.expand_dims(x_pred, axis=0)
    y_pred=num_to_char[np.argmax(model.predict(x_pred))]
    sys.stdout.write(y_pred)
    sequence+=y_pred
    sequence=sequence[1:]
    #print(sequence)
  
